# 🎬 Advanced Content-Based Movie Recommendation Engine (NLP)
**By: Vishwaraj Khatpe | Final Year Engineering Student (AIML)**

---

## 1. Executive Summary
This project addresses the "cold start" problem in recommendation systems—how to suggest movies to users without needing their historical viewing data. By leveraging Natural Language Processing (NLP) techniques on movie plot summaries, this engine identifies semantic similarities between films that simple genre-matching cannot capture.

---

## 2. Technical Approach
Instead of traditional metadata clustering, this system utilizes a Vector Space Model:

* **Text Ingestion:** Processing raw text from movie "overviews" (plot summaries).
* **TF-IDF Vectorization:** Converting unstructured text into a mathematical feature matrix, weighing unique plot points higher than common words.
* **Cosine Similarity:** Calculating the angular distance between movie vectors to quantify semantic relatedness.



---

## 3. Why This Approach?

* **Granularity:** While *Interstellar* and *Star Wars* are both "Sci-Fi", NLP can distinguish that one is about "black holes/physics" and the other is a "space opera."
* **Scalability:** Matrix-based filtering is highly optimized for large datasets.

### [Code] - Imports

In [1]:
# Core Libraries for Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Natural Language Processing & Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel # Faster than cosine_similarity for dense matrices

# Configuration
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

print(" Libraries Imported Successfully")

 Libraries Imported Successfully


## 4. Data Ingestion & Cleaning

We load the dataset and perform critical cleaning steps to prepare the text data for vectorization.

### Key Decisions:

* **Memory Management:** The dataset can be large, so we process only necessary columns.
* **Text Sanitization:** Null values in text columns cause vectorizers to crash. We must fill NaN values with empty strings.

### [Code] - Data Loading

In [2]:
# Load dataset
try:
    df = pd.read_csv("movies_metadata.csv", low_memory=False)
    print(f"Original Dataset Shape: {df.shape}")
except FileNotFoundError:
    print("❌ Error: 'movies_metadata.csv' not found.")

# Select distinct features
selected_cols = ['title', 'overview', 'genres', 'vote_average', 'popularity', 'vote_count']
movies = df[selected_cols].copy()

# --- OPTIMIZATION START ---
# Data Cleaning & Filtering
# 1. Clean numeric columns
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')
movies['vote_average'] = pd.to_numeric(movies['vote_average'], errors='coerce')
movies['vote_count'] = pd.to_numeric(movies['vote_count'], errors='coerce')

# 2. Drop duplicates and NaN titles
movies = movies.dropna(subset=['title'])
movies = movies.drop_duplicates(subset='title')

# 3. FILTERING: Keep only movies with at least 50 votes
# This removes obscure movies, reducing dataset size from ~45k to ~9k-12k high-quality movies
# This drastically reduces memory usage from 16GB to ~1GB
movies = movies[movies['vote_count'] >= 50]

# 4. Fill Null Plot Summaries
movies['overview'] = movies['overview'].fillna('')

# Reset index to ensure robust lookups later
movies = movies.reset_index(drop=True)

print(f"Optimized Dataset Shape: {movies.shape}")
# --- OPTIMIZATION END ---

movies[['title', 'vote_count', 'overview']].head(3)

Original Dataset Shape: (45466, 24)
Optimized Dataset Shape: (8548, 6)


,title,vote_count,overview
0,Toy Story,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,2413.0,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,92.0,A family wedding reignites the ancient feud be...


# **NLP Feature Engineering**


## 5. Feature Engineering: TF-IDF Vectorization

To compare plot summaries mathematically, we must convert text to numbers. We use **TF-IDF (Term Frequency-Inverse Document Frequency)**.

### Technical Insight: Why TF-IDF and not simple Word Counts?

* **Term Frequency (TF):** Counts how often a word appears in a specific movie plot.
* **Inverse Document Frequency (IDF):** Penalizes words that appear in every movie (like "the", "movie", "plot"). This ensures that generic words don't dominate the similarity score, allowing unique keywords (e.g., "wizard", "dinosaur", "heist") to drive recommendations.

### [Code] - TF-IDF Matrix Construction

In [4]:
# Initialize TF-IDF Vectorizer
# stop_words='english': Removes common words
# min_df=3: Ignores words that appear in fewer than 3 movies (removes typos/noise)
tfidf = TfidfVectorizer(stop_words='english', min_df=3)

print("🔄 Constructing TF-IDF Matrix...")
tfidf_matrix = tfidf.fit_transform(movies['overview'])

# OPTIMIZATION: Convert to float32 to reduce memory usage by 50%
tfidf_matrix = tfidf_matrix.astype('float32')

print(f"✅ TF-IDF Matrix Created. Shape: {tfidf_matrix.shape}")

🔄 Constructing TF-IDF Matrix...
✅ TF-IDF Matrix Created. Shape: (8548, 10732)


## 6. Computing Similarity

With our text converted to vectors, we calculate the similarity between every pair of movies.

We use the **Linear Kernel** (equivalent to Cosine Similarity for normalized data). This computes the cosine of the angle between two vectors:

* **Score = 1.0**: The plots are identical.
* **Score = 0.0**: The plots share no meaningful words.



> **Note:** The resulting matrix is massive ($N \times N$ movies), containing the similarity score of every movie against every other movie.

### [Code] - Cosine Similarity Matrix

In [5]:
print("🔄 Computing Cosine Similarity Matrix...")

# Compute the linear kernel (dot product)
# Because we filtered the data, this will now fit in your RAM
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print(f"✅ Similarity Matrix Computed. Shape: {cosine_sim.shape}")

# Create a reverse mapping of indices and movie titles
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

🔄 Computing Cosine Similarity Matrix...
✅ Similarity Matrix Computed. Shape: (8548, 8548)


# **Inference Engine**


## 7. Building the Recommendation Function

This function encapsulates the core logic of the system through a four-step process:

1.  **Lookup**: Find the unique index of the user's input movie title in the metadata DataFrame.
2.  **Similarity Retrieval**: Access the specific row of similarity scores for that movie from the `cosine_sim` matrix.
3.  **Ranking**: Sort all movies based on their similarity scores in descending order to identify the closest matches.
4.  **Fetch**: Filter out the input movie itself and return the **top 10** most similar movie titles.

---


> **Implementation Note:** Ensure that the input title is pre-processed or matched using a case-insensitive search to prevent `IndexErrors` during the lookup phase.

### [Code] - Recommendation Function

In [7]:
def get_recommendations(title, cosine_sim=cosine_sim):
    """
    Returns top 10 recommended movies based on plot similarity.
    """
    try:
        # 1. Get the index of the movie that matches the title
        # We use strict matching here; in production, we'd use fuzzy matching
        idx = indices[title]
        
        # Handle cases where multiple movies might have the same title
        if isinstance(idx, pd.Series):
            idx = idx.iloc[0]

        # 2. Get the pairwise similarity scores
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 3. Sort movies based on similarity scores (Highest first)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. Get top 10 similar movies (ignoring index 0, which is the movie itself)
        sim_scores = sim_scores[1:11]

        # 5. Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # 6. Return the top 10 most similar movies with their ratings
        return movies[['title', 'vote_average', 'overview']].iloc[movie_indices]
        
    except KeyError:
        return f"⚠️ Movie '{title}' not found in the filtered database (Must have >50 votes)."

# --- Test Case ---
test_movie = 'The Dark Knight Rises'
print(f"🎬 Recommendations for '{test_movie}':")
get_recommendations(test_movie)

🎬 Recommendations for 'The Dark Knight Rises':


,title,vote_average,overview
80,Batman Forever,5.2,The Dark Knight of Gotham City confronts a das...
4592,The Dark Knight,8.3,Batman raises the stakes in his war on crime. ...
320,Batman,7.0,The Dark Knight of Gotham City begins his war ...
5306,Batman: Under the Red Hood,7.6,Batman faces his ultimate challenge as the mys...
710,Batman Returns,6.6,"Having defeated the Joker, Batman now faces th..."
3628,Batman Beyond: Return of the Joker,7.5,"The Joker is back with a vengeance, and Gotham..."
7926,Batman: Bad Blood,6.8,Bruce Wayne is missing. Alfred covers for him ...
6094,"Batman: The Dark Knight Returns, Part 1",7.7,Batman has not been seen for ten years. A new ...
5697,Batman: Year One,7.1,Two men come to Gotham City: Bruce Wayne after...
7405,Batman Unlimited: Animal Instincts,5.5,Gotham City is under siege by a series of biza...


## **8. Interactive Demo (User Input)**
Run this cell to test the recommendation engine interactively. Type a movie name to get suggestions, or type 'exit' to stop the program.

In [13]:
# Interactive CLI (Command Line Interface) for testing
import sys

def interactive_recommender():
    print("🎥 ---------------------------------------------------")
    print("   MOVIE RECOMMENDATION ENGINE (NLP PROTOTYPE)")
    print("   Type 'exit' or 'quit' to stop the program.")
    print("--------------------------------------------------- 🎥")

    while True:
        try:
            # 1. Get user input
            user_input = input("\nEnter a movie you like: ").strip()
            
            # 2. Exit condition (Case insensitive)
            if user_input.lower() in ['exit', 'quit', 'end', 'stop']:
                print("\nExiting Recommendation Engine. Goodbye!")
                break
            
            # Skip empty inputs
            if not user_input:
                continue

            # 3. CASE-INSENSITIVE LOOKUP LOGIC
            # We look for the user's input in the 'title' column (lowercased)
            # If we find a match, we grab the *official* title from the database
            match = movies[movies['title'].str.lower() == user_input.lower()]

            if not match.empty:
                # Retrieve the actual title (e.g., convert 'avatar' -> 'Avatar')
                valid_title = match['title'].iloc[0]
                
                # Call the recommendation function with the correct title
                recommendations = get_recommendations(valid_title)
            else:
                # If no match found, set recommendations to an error string
                recommendations = f"Movie '{user_input}' not found in the filtered database."

            # 4. Display Results
            if isinstance(recommendations, str):
                print(f"     {recommendations}")
                print("      Tip: Check for typos. Remember we only have movies with >50 votes!")
            else:
                print(f"\nTop Recommendations for '{valid_title}':")
                print("-" * 60)
                
                # Format the output nicely
                for i, (index, row) in enumerate(recommendations.iterrows(), 1):
                    print(f"{i}. {row['title']} (Rating: {row['vote_average']}/10)")
                    
                    # Print a short snippet of the plot (truncate if too long)
                    plot_snippet = row['overview'][:100] + "..." if len(row['overview']) > 100 else row['overview']
                    print(f"   📖 {plot_snippet}\n")
                    
        except KeyboardInterrupt:
            # Handle user pressing Ctrl+C gracefully
            print("\n\n Program interrupted. Exiting...")
            break
        except Exception as e:
            print(f" An unexpected error occurred: {e}")

# Run the interactive system
interactive_recommender()

🎥 ---------------------------------------------------
   MOVIE RECOMMENDATION ENGINE (NLP PROTOTYPE)
   Type 'exit' or 'quit' to stop the program.
--------------------------------------------------- 🎥

Top Recommendations for 'Iron Man':
------------------------------------------------------------
1. Clown (Rating: 5.3/10)
   📖 A loving father finds a clown suit for his son's birthday party, only to realize the suit is part of...

2. Iron Man 2 (Rating: 6.6/10)
   📖 With the world now aware of his dual life as the armored superhero Iron Man, billionaire inventor To...

3. Iron Man 3 (Rating: 6.8/10)
   📖 When Tony Stark's world is torn apart by a formidable terrorist called the Mandarin, he starts an od...

4. Avengers: Age of Ultron (Rating: 7.3/10)
   📖 When Tony Stark tries to jumpstart a dormant peacekeeping program, things go awry and Earth’s Mighti...

5. Cradle 2 the Grave (Rating: 5.8/10)
   📖 Gang leader Tony pulls off a major diamond heist with his crew, but cop-turned-crimin

# **Conclusion**

## 8. Conclusion & Future Improvements

### Project Summary
We successfully built a **Content-Based Filtering System** using Natural Language Processing (NLP). By processing plot summaries with **TF-IDF**, we enabled the system to "read" movie descriptions and understand semantic relationships, providing recommendations that go deeper than simple genre matching.

---

#### **Key Technical Achievements**
1.  **NLP Pipeline:** Implemented **TF-IDF Vectorization** to convert unstructured text data into a high-dimensional feature space, enabling mathematical comparison of movie plots.
2.  **Performance Optimization:** Addressed critical **Memory Constraints (OOM Errors)** inherent in $O(N^2)$ distance matrices. By intelligently filtering for "high-quality" movies (Votes $\ge$ 50) and utilizing `float32` data types, we reduced memory usage by over **90%** while simultaneously improving recommendation relevance.
3.  **Interactive Inference:** Moved beyond static script execution by building a **Command-Line Interface (CLI)**, allowing users to query the model dynamically in real-time.

---

## Future Enhancements (Interview Talking Points)
To scale this project for a production environment, I would consider the following improvements:

### 1. Hybridization
Combining this **Content-Based** score with a **Collaborative Filtering (SVD)** model to incorporate user personalization and historical behavior.

### 2. Advanced Embeddings
Replacing TF-IDF with **BERT** or **Word2Vec** embeddings. This allows the model to capture deeper context (e.g., understanding the semantic relationship between terms like "king" and "queen").

### 3. Approximate Nearest Neighbors (ANN)
For datasets with millions of movies, calculating the full Cosine Similarity matrix is computationally expensive. Using libraries like **FAISS** or **Annoy** can reduce inference time:

* **Current complexity:** $O(N)$
* **Optimized complexity:** $O(\log N)$

### 4. Deployment:
Containerize the application using **Docker** and expose the inference engine via a **FastAPI** endpoint.

